In [2]:
import sys
!{sys.executable} -m pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/ce/85/62825cabc6aad53104b7b6d12eb2ad74737d268630032d07b74d4444cb72/psycopg2_binary-2.9.9-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.8 MB/s eta 0:00:00a 0:00:01


In [28]:
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="corso-kg-postgres",
    database="corso-kg-postgres",
    user="postgres",
    password="kgCefriel"
)

# Create a cursor object to interact with the database
cur = conn.cursor()
cur

<cursor object at 0x7fc7a5e22b60; closed: 0>

In [29]:
# Query to get the list of tables in the database
tables_query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name
"""

# Execute the tables query
cur.execute(tables_query)
tables = cur.fetchall()

# Iterate over the tables
for table in tables:
    table_name = table[0]
    print("Table:", table_name)

    # Query to get 3 results for each table to check the columns
    columns_query = f"""
        SELECT *
        FROM {table_name}
        LIMIT 3
    """

    # Execute the columns query
    cur.execute(columns_query)
    columns = [desc[0] for desc in cur.description]

    print("Columns:")
    print(columns)

    print("")

Table: assignment
Columns:
['employee_id', 'competence_id', 'level', 'interest']

Table: competences
Columns:
['id', 'dimension', 'competence', 'description']

Table: employees
Columns:
['id', 'name', 'surname', 'unit']



In [31]:
# Execute query and print the result
def execute_query(cur, query):
    # Execute the query
    cur.execute(query)
    custom_results = cur.fetchall()
    # Get the column names
    columns = [desc[0] for desc in cur.description]

    # Print the custom query results as a table
    for col in columns:
        print(col, end="\t")
    print()
    for row in custom_results:
        for col in row:
            print(col, end="\t")
        print()

In [32]:
# Query: Competences of an Employee
name = "Jane"
surname = "Brown"
employee_competences = f"""
    SELECT competence_id, competence
    FROM employees
    JOIN assignment ON employees.id = assignment.employee_id
    JOIN competences ON competences.id = assignment.competence_id
    WHERE employees.name = '{name}' AND employees.surname = '{surname}'
"""

execute_query(cur, employee_competences)

competence_id	competence	
4.4	Protecting the environment	
4.3	Protecting health and well-being	
4.2	Protecting personal data and privacy	
3.4	Programming	
1.3	Managing data, information and digital content	
3.3	Copyright and licences	
5.3	Creatively using digital technologies	
5.1	Solving technical problems	
2.1	Interacting through digital technologies	
2.5	Netiquette	


In [33]:
# Query: Employees with competences similar to the one of another employee
name = "Jane"
surname = "Brown"
similar_employees = f"""
    WITH jane_competences AS (
        SELECT competence_id
        FROM employees
        JOIN assignment ON employees.id = assignment.employee_id
        WHERE employees.name = '{name}' AND employees.surname = '{surname}'
    )
    SELECT employees.id, employees.name, employees.surname, COUNT(assignment.competence_id) AS num_competences
    FROM employees
    JOIN assignment ON employees.id = assignment.employee_id
    WHERE assignment.competence_id IN (SELECT competence_id FROM jane_competences) 
        AND NOT(employees.name = '{name}' AND employees.surname = '{surname}')
    GROUP BY employees.id, employees.name, employees.surname
    ORDER BY num_competences DESC;
"""

execute_query(cur, similar_employees)

id	name	surname	num_competences	
EMPLOYEE_23	Jane	Taylor	10	
EMPLOYEE_4	David	Brown	10	
EMPLOYEE_16	David	Davis	9	
EMPLOYEE_8	David	Miller	9	
EMPLOYEE_22	Sophia	Johnson	9	
EMPLOYEE_6	Jane	Davis	8	
EMPLOYEE_29	Daniel	Brown	8	
EMPLOYEE_20	Sophia	Harris	7	
EMPLOYEE_19	David	Harris	7	
EMPLOYEE_2	David	Brown	7	
EMPLOYEE_10	Michael	Wilson	7	
EMPLOYEE_12	David	Johnson	7	
EMPLOYEE_3	Olivia	Davis	6	
EMPLOYEE_15	Emma	Johnson	6	
EMPLOYEE_9	Andrew	Wilson	5	
EMPLOYEE_13	Daniel	Davis	5	
EMPLOYEE_30	Emma	Clark	5	
EMPLOYEE_27	Sophia	Davis	4	
EMPLOYEE_21	Olivia	Clark	4	
EMPLOYEE_17	David	Miller	4	
EMPLOYEE_28	Michael	Johnson	4	
EMPLOYEE_5	David	Taylor	3	
EMPLOYEE_1	Jane	Smith	3	
EMPLOYEE_24	Olivia	Wilson	2	
EMPLOYEE_18	Emily	Brown	1	
EMPLOYEE_25	Emily	Wilson	1	


In [26]:
# Close the cursor and the connection
cur.close()
conn.close()